In [2]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
from geopy.geocoders import Nominatim

# List of all 50 states
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", 
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", 
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", 
    "Nevada", "New-Hampshire", "New-Jersey", "New-Mexico", "New-York", "North-Carolina", "North-Dakota", "Ohio", 
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode-Island", "South-Carolina", "South-Dakota", "Tennessee", "Texas", 
    "Utah", "Vermont", "Virginia", "Washington", "West-Virginia", "Wisconsin", "Wyoming"
]

# Initialize an empty list to hold all the data
all_data = []

# Initialize the geolocator
geolocator = Nominatim(user_agent="real_estate_scraper", timeout=10)

# Function to get latitude and longitude
def get_lat_lng(city, state):
    try:
        location = geolocator.geocode(f"{city}, {state}")
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error geocoding {city}, {state}: {e}")
    return None, None

# Loop through each state
for state in states:
    url = f"https://www.redfin.com/state/{state}"  # Update the URL with the current state
    
    try:
        # Send a GET request to the webpage
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the table based on its class name
        table = soup.find('table', {'class': 'filterableTable table-styles-basic'})
        
        # Skip if the table is not found
        if not table:
            print(f"No data found for {state}")
            continue

        # Extract the rows from the table body
        rows = table.find('tbody').find_all('tr')

        # Parse and extract data from each row
        for row in rows:
            columns = row.find_all('td')
            row_data = [col.get_text(strip=True) for col in columns]
            if len(row_data) < 3:  # Skip rows with insufficient data
                continue
            city = row_data[0].split(',')[0].strip()
            avg_list_price = row_data[1]
            avg_price_sqft = row_data[2]
            avg_days_on_market = row_data[3]
            lat, lng = get_lat_lng(city, state)  # Add geocoding here
            all_data.append([state, city, avg_list_price, avg_price_sqft, avg_days_on_market, lat, lng])
        
        print(f"Data successfully extracted for {state}")
    except Exception as e:
        print(f"Error extracting data for {state}: {e}")

# Check if any data was collected
if all_data:
    # Define the column names
    column_names = ["state", "city", "avg.list price", "avg.price/sqft", "avg.days on market", "lat", "lng"]

    # Create a pandas DataFrame
    df = pd.DataFrame(all_data, columns=column_names)

    # Clean and save the data
    df['avg.list price'] = pd.to_numeric(df['avg.list price'].str.replace('[$,]', '', regex=True),errors='coerce')
    df['avg.price/sqft'] = pd.to_numeric(df['avg.price/sqft'].str.replace('[$,]', '', regex=True), errors='coerce')
    df['avg.days on market'] = pd.to_numeric(df['avg.days on market'], errors='coerce')

    # Save the data to the "Data" directory
    base_path = os.getcwd()
    output_csv_path = os.path.join(base_path, "Data", "cleaned_redfin_data_with_coords.csv")
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    print(f"Data saved to {output_csv_path}")
else:
    print("No data was extracted.")

Data successfully extracted for Alabama
Data successfully extracted for Alaska
Data successfully extracted for Arizona
Data successfully extracted for Arkansas
Data successfully extracted for California
Data successfully extracted for Colorado
Data successfully extracted for Connecticut
Data successfully extracted for Delaware
Data successfully extracted for Florida
Data successfully extracted for Georgia
Data successfully extracted for Hawaii
Data successfully extracted for Idaho
Data successfully extracted for Illinois
Data successfully extracted for Indiana
Data successfully extracted for Iowa
Data successfully extracted for Kansas
Data successfully extracted for Kentucky
Data successfully extracted for Louisiana
Data successfully extracted for Maine
Data successfully extracted for Maryland
Data successfully extracted for Massachusetts
Data successfully extracted for Michigan
Data successfully extracted for Minnesota
Data successfully extracted for Mississippi
Data successfully extr

In [4]:
df.head(50)

,state,city,avg.list price,avg.price/sqft,avg.days on market,lat,lng
0,Alabama,1.Huntsville,374500.0,162.0,91.0,34.729847,-86.585901
1,Alabama,2.Birmingham,165000.0,90.0,82.0,33.513400,-86.805721
2,Alabama,3.Mobile,250590.0,124.0,85.0,30.691346,-88.043751
3,Alabama,4.Madison,541725.0,177.0,108.0,34.692873,-86.751143
4,Alabama,5.Montgomery,190000.0,108.0,81.0,32.377420,-86.309168
5,Alabama,6.Gulf Shores,514500.0,285.0,131.0,30.271122,-87.689383
6,Alabama,7.Hoover,514950.0,172.0,86.0,32.541844,-86.138386
7,Alabama,8.Fairhope,622245.0,214.0,108.0,30.523696,-87.910950
8,Alabama,9.Orange Beach,745000.0,442.0,134.0,30.294368,-87.573594
9,Alabama,10.Athens,349900.0,155.0,103.0,33.601743,-85.959151


In [6]:
# Load the CSV file
file_path = os.path.join(base_path, "Data", "cleaned_redfin_data_with_coords.csv")
data = pd.read_csv(file_path)

In [8]:
# Clean the city column to extract proper city names
data['city'] = data['city'].str.extract(r'\d+\.(.+)')[0].str.strip()

In [10]:
# Replace problematic values ('—') and clean numerical columns
columns_to_clean = ['avg.list price', 'avg.price/sqft', 'avg.days on market']
for col in columns_to_clean:
    data[col] = data[col].replace({r'\$': '', r',': '', '—': None}, regex=True)
    data[col] = pd.to_numeric(data[col], errors='coerce')

In [12]:
# Save the cleaned DataFrame to a CSV file
data.to_csv(output_csv_path, index=False)
print(f"Final cleaned data preview:")
print(data.head())

Final cleaned data preview:
     state        city  avg.list price  avg.price/sqft  avg.days on market  \
0  Alabama  Huntsville        374500.0           162.0                91.0   
1  Alabama  Birmingham        165000.0            90.0                82.0   
2  Alabama      Mobile        250590.0           124.0                85.0   
3  Alabama     Madison        541725.0           177.0               108.0   
4  Alabama  Montgomery        190000.0           108.0                81.0   

         lat        lng  
0  34.729847 -86.585901  
1  33.513400 -86.805721  
2  30.691346 -88.043751  
3  34.692873 -86.751143  
4  32.377420 -86.309168  
